In [1]:
import pandas as pd

# read results files from icsd and springer

In [2]:
icsd = pd.read_csv('../Data Files/icsd_results.csv')
springer = pd.read_csv('../Data Files/springer_results.csv')

combined = icsd.copy(deep=True)

### define a useful helper function

In [3]:
def jagged_concatenate(arr1,arr2):
    concatenated = []
    for i in range(len(arr1)):
        concatenated += [arr1[i] + arr2[i]]
    
    return concatenated

# find any discrepancies between the two sources

In [4]:
for i, row in icsd.iterrows():
    icsd_comp = row['Composition']
    springer_matches = springer.loc[springer['Composition'] == icsd_comp]
    
    if len(springer_matches) == 0: # icsd comp doesn't exist in springer... keep the icsd comp
        continue 
    else:
        icsd_sg = str(row['Space Groups'])
        springer_sg = str(eval(str(springer_matches['Space Groups'].values))[0])
        icsd_ptypes = str(row['Prototype'])
        springer_ptypes = str(eval(str(springer_matches['Prototype'].values))[0])
        icsd_filenames = str(row['Filename'])
        springer_filenames = str(eval(str(springer_matches['Filename'].values))[0])
        
        if icsd_sg != springer_sg: # if space groups do not match, add space group and prototypes to the icsd entry
            combined_sg = str(['SG CONFLICT', 'SG CONFLICT'])
            combined_ptypes = str([['SG CONFLICT']])
            combined_filenames = str([['SG CONFLICT']])
                        
            
        elif icsd_ptypes != springer_ptypes: # if space groups do match, but prototypes don't match
            combined_sg = icsd_sg
            if len(eval(icsd_sg)) == 1: # if single space group
                combined_ptypes = f'[{str(list(set(eval(icsd_ptypes)[0] + eval(springer_ptypes)[0])))}]' 
            else: # if multiple space groups
                icsd_ptypes = eval(icsd_ptypes)
                springer_ptypes = eval(springer_ptypes)
                combined_ptypes = str(jagged_concatenate(icsd_ptypes,springer_ptypes))
            combined_filenames = jagged_concatenate(eval(icsd_filenames),eval(springer_filenames))
            
        else: # if everything matches
            combined_sg = icsd_sg
            combined_ptypes = icsd_ptypes
            combined_filenames = jagged_concatenate(eval(icsd_filenames),eval(springer_filenames))
        
        # update values in final dataframe
        combined.at[i,'Space Groups'] = combined_sg
        combined.at[i,'Prototype'] = combined_ptypes
        combined.at[i,'Filename'] = str(combined_filenames)    
        
        # update error code
        
        # determine the error code
        ## 0 or 3 = single space group, single prototype
        ## 1 or 4 = single space group, multiple prototypes
        ## 2 or 5 = multiple space groups, (multiple prototypes)
        ## 6 = other error (there shouldn't be any 6's)
        ## 7 = .cif file(s) couldn't be opened (FATAL)
        ## 8 = space group conflict
    
        if len(eval(combined_sg)) == 1: 
            space_group_conflicts = False
            if len(eval(combined_ptypes)[0]) == 1: single_prototype = True
            else: single_prototype = False
    
        else: space_group_conflicts = True

        if not space_group_conflicts and single_prototype: error_code = 0
        elif not space_group_conflicts and not single_prototype: error_code = 1
        elif space_group_conflicts: error_code = 2
        elif combined_ptypes == 'Error': error_code = 7
        else: error_code = 6
        
        combined.at[i,'Error Code'] = error_code
        
        # remove matches from springer dataframe
        springer = springer.drop([springer_matches.index[0]])
            
# finally, append all remaining springer entries that remain

combined = pd.concat([combined,springer],ignore_index=True)
combined = combined.drop(columns = ['Unnamed: 0'])

# save combined csv

In [5]:
combined.to_csv('../Data Files/combined_results.csv')